# UK House Prices on Map
### Data Engineering Capstone Project

#### Project Summary

This project's scope is to develop a data lake of house sale transactions in the UK.
It is designed with data analysts in mind, who need to visualise temporal changes in house prices in different regions of the UK. Currently, the data warehouse combines two data sources: price paid data transactions received at Land Registry during the last 15 years (25M rows), as well as geographical and administrative metadata about UK post codes (1.7M rows).

In [1]:
import configparser
import os
import sys

import pandas as pd
from pyspark.sql import SparkSession

from spark_jobs import etl_spark

# Make the local module discoverable:
sys.path.append(os.path.join(os.getcwd(), 'spark_jobs'))


!{sys.executable} -m pip install utm pyspark pandas psycopg2-binary

### Step 1: Scope the Project and Gather Data

#### Scope 
The goal of the project is to provide data analysts (e.g. government statisticians preparing annual reports, business analytics in real estate companies using dashboards) with a data platform that focuses on time series of prices and geographical location.
For that, we need a dataset of sale transactions and map locations of UK addresses, which we combine from two different open sources.
Since the solution needs to allow end users perform complex queries (e.g. analyses by year and location), and the dataset is being continuously generated with substantial size, AWS Redshift is used as a platform for queries, together with Apache Spark for data processing.

An example query: "Visualize the change in average house prices on a UK map".

#### Describe and Gather Data 
Describe the data sets you're using. Where did it come from? What type of information is included? 

Currently, we use two data sources:
1. **Price Paid Dataset (PPD)** that includes standard and additional price paid data transactions received at UK Land Registry from 1 January 1995 to 1 May 2020. **25,233,169 rows.** Download link: http://prod.publicdata.landregistry.gov.uk.s3-website-eu-west-1.amazonaws.com/pp-complete.csv
    - *Contains HM Land Registry data © Crown copyright and database right 2020. This data is licensed under the Open Government Licence v3.0.*
    - The dataset is being updated once a month (useful for maintenance) and is also available for download as a single file.
    
Data item | Explanation (where appropriate)
----------|--------------------------------
Transaction unique identifier | A reference number which is generated automatically recording each published sale. The number is unique and will change each time a sale is recorded.
Price | Sale price stated on the transfer deed.
Date of Transfer | Date when the sale was completed, as stated on the transfer deed.
Postcode | This is the postcode used at the time of the original transaction. Note that postcodes can be reallocated and these changes are not reflected in the Price Paid Dataset.
Property Type | D = Detached, S = Semi-Detached, T = Terraced, F = Flats/Maisonettes, O = Other
Old/New | Indicates the age of the property and applies to all price paid transactions, residential and non-residential. Y = a newly built property, N = an established residential building
Duration | Relates to the tenure: F = Freehold, L= Leasehold etc. Note that HM Land Registry does not record leases of 7 years or less in the Price Paid Dataset.
PAON | Primary Addressable Object Name. Typically the house number or name.
SAON | Secondary Addressable Object Name. Where a property has been divided into separate units (for example, flats), the PAON (above) will identify the building and a SAON will be specified that identifies the separate unit/flat.
Street	
Locality	
Town/City	
District	
County	
PPD Category Type | Indicates the type of Price Paid transaction. A = Standard Price Paid entry, includes single residential property sold for value. B = Additional Price Paid entry including transfers under a power of sale/repossessions, buy-to-lets (where they can be identified by a Mortgage) and transfers to non-private individuals.
Record Status	| Monthly file only. Indicates additions, changes and deletions to the records. A = Addition. C = Change. D = Delete.

2. **Code-Point Open dataset** that has administrative and geographical metadata about all UK postal codes. **1,706,904 rows.** Download link: https://www.ordnancesurvey.co.uk/business-government/products/code-point-open (needs email address).
    - *Contains OS data © Crown copyright and database right 2018*
    - *Contains Royal Mail data © Royal Mail copyright and database right 2018*
    - *Contains National Statistics data © Crown copyright and database right 2018*

Data item | Explanation (where appropriate)
----------|--------------------------------
Postcode
Positional quality indicator | Indicates the quality of the data underlying the Code-Point Open location coordinate (CPLC).
Eastings | Eastward-measured British National Grid (BNG) geographic Cartesian coordinate for a point. 
Northings | Northward-measured British National Grid (BNG) geographic Cartesian coordinate for a point. 
Country code | Either England, Scotland or Wales.
NHS regional HA code | The National Health Service (NHS) region.
NHS HA code
Admin county code | The local government county code.
Admin district code | The local government district code.
Admin ward code | The local government ward code.

### Step 2: Explore and Assess the Data
#### Explore the Data 
Since both of the data sources contain government data, there are almost no issues with data quality - moreover, the UK Land Registry publishes error corrections in a timely manner. The same applies to the postcodes dataset, which is being updated in case of errors by its author company.

In [2]:
import os

config = configparser.ConfigParser()
config.read("credentials.cfg")

PPD_PATH = config.get("S3", "SPARK_INPUT_PPD")
PPD_HEADERS = config.get("S3", "SPARK_INPUT_PPD_HEADERS")
POSTCODE_PATH = config.get("S3", "SPARK_INPUT_POSTCODES")
REGION_NAMES_PATH = config.get("S3", "SPARK_INPUT_REGION_NAMES")

os.environ["AWS_ACCESS_KEY_ID"] = config.get("AWS", "AWS_ACCESS_KEY_ID")
os.environ["AWS_SECRET_ACCESS_KEY"] = config.get("AWS", "AWS_SECRET_ACCESS_KEY")

spark = (
    SparkSession.builder.config(
        "spark.jars.packages", "org.apache.hadoop:hadoop-aws:2.7.0"
    )
    .enableHiveSupport()
    .getOrCreate()
)

##### Price Paid Dataset

In [3]:
%%time
ppd_df = etl_spark.read_ppd_table(spark, PPD_PATH, PPD_HEADERS)
ppd_df.printSchema()

root
 |-- transaction_unique_identifier: string (nullable = true)
 |-- price: string (nullable = true)
 |-- date_of_transfer: string (nullable = true)
 |-- postcode: string (nullable = true)
 |-- property_type: string (nullable = true)
 |-- old_new: string (nullable = true)
 |-- duration: string (nullable = true)
 |-- paon: string (nullable = true)
 |-- saon: string (nullable = true)
 |-- street: string (nullable = true)
 |-- locality: string (nullable = true)
 |-- town_city: string (nullable = true)
 |-- district: string (nullable = true)
 |-- county: string (nullable = true)
 |-- ppd_category_type: string (nullable = true)
 |-- record_status: string (nullable = true)

CPU times: user 9.91 ms, sys: 4.22 ms, total: 14.1 ms
Wall time: 4.48 s


In [4]:
ppd_df.head()

Row(transaction_unique_identifier='{A42E2F04-2538-4A25-94C5-49E29C6C8FA8}', price='18500', date_of_transfer='1995-01-31 00:00', postcode='TQ1 1RY', property_type='F', old_new='N', duration='L', paon='VILLA PARADISO', saon='FLAT 10', street='HIGHER WARBERRY ROAD', locality='TORQUAY', town_city='TORQUAY', district='TORBAY', county='TORBAY', ppd_category_type='A', record_status='A')

Number of rows:

In [5]:
%%time
ppd_df.count()

CPU times: user 3.27 ms, sys: 3.07 ms, total: 6.34 ms
Wall time: 18.9 s


25233169

##### Postcodes Dataset

In [6]:
post_df = etl_spark.read_postcodes(spark, POSTCODE_PATH)
post_df.printSchema()

root
 |-- postcode: string (nullable = true)
 |-- positional_quality_indicator: string (nullable = true)
 |-- eastings: string (nullable = true)
 |-- northings: string (nullable = true)
 |-- country_code: string (nullable = true)
 |-- nhs_regional_ha_code: string (nullable = true)
 |-- nhs_ha_code: string (nullable = true)
 |-- admin_county_code: string (nullable = true)
 |-- admin_district_code: string (nullable = true)
 |-- admin_ward_code: string (nullable = true)



In [7]:
post_df.head()

Row(postcode='B1  1AY', positional_quality_indicator='10', eastings='406523', northings='286448', country_code='E92000001', nhs_regional_ha_code='E19000001', nhs_ha_code='E18000005', admin_county_code=None, admin_district_code='E08000025', admin_ward_code='E05011151')

Number of rows:

In [8]:
%%time
post_df.count()

CPU times: user 1.06 ms, sys: 1.52 ms, total: 2.58 ms
Wall time: 1.04 s


1706904

#### Cleaning Steps
There are a few steps we need to undertake to clean the datasets:

1. Making postcodes consistent in both datasets, as it is our join key. The simplest approach is removing white spaces and uppercasing the text (see `utils._normalise_postcode`).
2. Filtering out "additional" transactions, i.e. those that have PPD type "B", as they seem to be transactions of a different nature (not sales).
2. Cleaning address in the sales dataset. For the chosen analysis level, the granular address of a property is not important and is used for archiving purposes only. Therefore, one solution is to collapse primary and secondary lines of address with the street name (see `utils._normalise_address`).
3. Translating abbreviations into region/country names. This is done to assist the end users to easier work with the database (see `utils._add_country_name`, `utils._add_region_name`).
4. Converting British National Grid (BNG) eastings and northings into latitude and longitude, since many map visualizations use those instead of BNG (see `utils._add_location`).

### Step 3: Define the Data Model
#### 3.1 Conceptual Data Model
While designing a data model for the project, it was important to decide which level of analysis users would perform. It could be property-level, and thus a property would be modeled individually based on its unique address. It could be area-level, where properties would be aggregated based on postal code. At the data lake stage, we choose the former approach, saving details of both postcodes and individual addresses.

For this project and its end user needs, a **star schema** seems to be most appropriate:
1. The data is highly structured, so a relational database/SQL approach is viable, providing high performance for JOIN queries needed for this type of analysis. The large size of the dataset can be mitigated by using scalable solutions like AWS Redshift.
2. The data naturally falls into facts/dimensions division: we have sales and information about each element (dimensions) of a sale.

Below is the selected target data model, with `Sale` being the fact table with `Time`, `Property`, `Postcode` dimensions:

<img src="uml.png" alt="UML Diagram of Star Schema" width="500"/>

#### 3.2 Mapping Out Data Pipelines
To transform the raw datasets into the chosen data model, we use the following pipeline:
1. Read the CSV files into Apache Spark dataframes.
2. Apply the cleaning/transformation steps, extract fact and dimension tables into separate parquet files.
3. Read the parquet files into Redshift.

### Step 4: Pipelines

Airflow is used to orchestrate the pipelines:

<img src="dag_flow.png" alt="Airflow DAG" width="100%"/>

#### 4.1 Create the data model

In [9]:
%%time
ppd_tables = etl_spark.get_ppd_tables(spark, PPD_PATH, PPD_HEADERS)
postcode_tables = etl_spark.get_postcode_tables(spark, POSTCODE_PATH, REGION_NAMES_PATH)

CPU times: user 153 ms, sys: 33.6 ms, total: 186 ms
Wall time: 1.64 s


In [10]:
def show_info(df):
    df.printSchema()
    df.show(5)

In [11]:
for table, _df in ppd_tables.items():
    print(table)
    show_info(_df)
    print()

property
root
 |-- property_address: string (nullable = true)
 |-- property_type: string (nullable = true)
 |-- is_new: boolean (nullable = true)
 |-- duration: string (nullable = true)

+--------------------+-------------+------+---------+
|    property_address|property_type|is_new| duration|
+--------------------+-------------+------+---------+
|A'BECKET GARDENS; 46|semi-detached| false| freehold|
|AALBORG PLACE; MI...|         flat|  true|leasehold|
|   AALTEN AVENUE; 22|     detached| false| freehold|
|    ABBATT CLOSE; 57|     terraced| false| freehold|
|    ABBAY STREET; 27|     terraced| false| freehold|
+--------------------+-------------+------+---------+
only showing top 5 rows


time
root
 |-- date: date (nullable = true)
 |-- year: integer (nullable = true)
 |-- month: integer (nullable = true)
 |-- day: integer (nullable = true)
 |-- week: integer (nullable = true)
 |-- weekday: integer (nullable = true)

+----------+----+-----+---+----+-------+
|      date|year|month|day|

In [12]:
for table, _df in postcode_tables.items():
    print(table)
    show_info(_df)
    print()

postcode
root
 |-- postcode: string (nullable = true)
 |-- district: string (nullable = true)
 |-- country: string (nullable = true)
 |-- longitude: float (nullable = true)
 |-- latitude: float (nullable = true)

+--------+----------+-------+----------+---------+
|postcode|  district|country| longitude| latitude|
+--------+----------+-------+----------+---------+
|   B11AY|Birmingham|England|-3.8408782|2.5912867|
|   B11BA|Birmingham|England|-3.8377116|2.5907912|
|   B11BB|Birmingham|England|-3.8400989| 2.596163|
|   B11BD|Birmingham|England| -3.839387|   2.5943|
|   B11BE|Birmingham|England|-3.8386576|2.5929344|
+--------+----------+-------+----------+---------+
only showing top 5 rows




In [13]:
spark.stop()

#### 4.2 Data Quality Checks
Data quality checks are governed by the corresponding Airflow operator that currently ensures that the tables are not empty. Moreover, there are integrity constraints in Redshift, like `NOT NULL` and data type checks.

#### 4.3 Data dictionary 

##### Sale

Data item | Explanation
----------|-------------------------------
id | Unique identifier of a sale transaction, the same as used in the Price Paid Dataset.
price | Price paid in the transaction, in British Pounds.
date | Date of the transaction.
property_id | Numeric ID of the property type being sold, corresponds to IDs in the Property table.
postcode | UK postcode of the property being sold.
address | Additional locality information about the concrete property being sold.

##### Postcode
Data item | Explanation
----------|-------------------------------
code | Postal code, uppercase with no whitespaces.
district | District name where the postcode is located.
country | Country name where the postcode is located.
latitude | Latitude of the postcode.
longitude | Longitude of the postcode.

##### Time
Data item | Explanation
----------|-------------------------------
date | Date.
day | Day of the date.
week | Week of the year in the date.
month | Month of the date.
year | Year of the date.
weekday | Day of the week in the date.

##### Property
Data item | Explanation
----------|-------------------------------
id | Unique property type id.
type | Property type (e.g. flat, detached).
is_new | Whether the property is being sold for the first time.
duration | Whether this is a leased or owned property.

#### Step 5: Conclusion (Project Write Up)

The project's goal was to enable data analysts to perform complex queries on a data warehouse containing information about property sales in the UK, as well as the geographical location of the houses. During the project scoping stage, the data sources were identified: 15 years of Price Paid Dataset entries and all UK postal code locations, both summing to 26M rows. A target data model was defined using a star schema, where house sales constitued the fact table, with time, postcodes, and property details being dimensions. To achieve non-trivial transformations, raw data was first put into a data lake and undergone processing before being copied into the final database.

The following tools and technologies were selected for the project:
- AWS Redshift: Since the data is already highly structured, it was reasonable to leverage SQL solutions to address the need of JOINs in potential analyses. The large size of datasets is mitigated by Redshift's scalability.
- Apache Spark: Used to additionally transform data before uploading it into Redshift. Spark was chosen to do the "heavy-lifting" of processing, since the transformations included non-trivial functions like converting BNG coordinates into latitude and longitude.
- All files were stored in AWS S3 to ensure the highest network speed between AWS tools.
  
The sales data was planned to be updated monthly, following the original price dataset's release schedule. The postal code data might be updated either ad-hoc after news that postcodes have been reallocated, or once a year as a routine maintenance (since such events happen relatively rarely).

The following scenarios were considered:
- If the data was to increase by 100x, it would still be possible to store it in Redshift, especially if the cluster size is increased.
- If the data populated a dashboard that must be updated on a daily basis by 7am every day, Airflow could be set to update the data daily before 7am.
- If the database needed to be accessed by 100+ people, Redshift would need to be scaled, which is also doable in AWS.